In [8]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import requests
import time

In [9]:
# custom_headers = {
#     "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7", 
#     "Accept-Encoding": "gzip, deflate, br", 
#     "Accept-Language": "en-US,en;q=0.9", 
#     "Upgrade-Insecure-Requests": "1", 
#     "Referer": "https://www.google.com/",
#     "User-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:126.0) Gecko/20100101 Firefox/126.0",
# }

cookies = {
    'session-id': '257-8319112-8619264',
    'session-id-time': '2082787201l',
    'i18n-prefs': 'GBP',
    'csm-hit': 'tb:s-G5H91RDPPJR06AWHKGPV^|1718214174313&t:1718214175053&adb:adblk_no',
    'ubid-acbuk': '257-0074823-8137474',
    'session-token': 'kXNGv7BpiPWBRYCaVulTNXwIGIgRRYF20sBKq3CPZiPG564Tj0IAto4dJl6M+/8i6qMxoPmRYg4yAi4JkeQYf0czFbp73lToOVpk0x7F/VssnK/vHGssCqKrMUeCyQxToAn5c1U6Z2k/3T4t0LXYEv1oBWQv2n1B+Vg3OFwVxoKmr+lJuH9kmidNrKWOlGg3sEWjzcgg3EPqnUcrKc+gb2BIWm8HMa7AJTLiQY2JaAp7637EmEaIz33d76AQNvd6RpJlcnSncbttchYk/uV9t4Vu4A7VNPz195zgz3DlNFGaR6sLxekDYZkazKHnlbsEzJk+YldKHrpwm/Pvyobw7cvdX+yM3l4NrMDYsbLFL5qziTA1xWqA+d1O2fM+YqWe',
    'lc-acbuk': 'en_GB',
    'x-acbuk': 'oG1gXi8lxjwAd7nRdiRn2^@lMPw0fNtkb7mYGJeXJby3HKqndGmQBY6c5odsU05cO',
    'at-acbuk': 'Atza^|IwEBIJ6LCCvQopCqxly9tASJoC7JkzDlA3mywQP4y_vd5W7K2OmV8ZzaOJZB1HpsRH72hd317Ccwhnxk9-JnNYmW63wFWyb4C_I7kztmPuO22UTmCh8adebNqjwMzj8brXfZLjwGCqcn9vW6PcCyORhL27v0XhWclQiEIgrcN4K84-viUpRmbgrJbpqjr0Ps5QRSFbR96eojQ04zCh2rWcR0piwpITPDPw7wyCk1cMdOSynnSQ',
    'sess-at-acbuk': 'Hh8/nyYMikZB9gcdfEFhOcCSNCVvzV89W1dpiUY+XBA=',
    'sst-acbuk': 'Sst1^|PQFcImeUnIA5G9cupIN6hp2pCXPKfOT802zYxeCduQjiFgjxaOZVZcn-1a_wbDewShq3A5rzwOMpXulty-o0qtl1ZKY-tbK2_lKroEipsdtM4bZcu1qE-dm-jsSgawjJ9I5aEAib_pLmuCeyFagkbiZRWXG_PiOtLt6356uAHR4RfH6QQUuzdchmtlADGHz9xnPu4d4xpeFv3X84HFrTOFd1gP-5HkPUCFQbKCJzzVgVMBxEad1PRo3mMBQwXnLl1LCr-07hUjBIaPII8k6q_01NSCGRXkRWJHcpS4jV_EpWM-E',
    'sp-cdn': 'L5Z9:IN',
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:126.0) Gecko/20100101 Firefox/126.0',
    'Accept': 'image/avif,image/webp,*/*',
    'Accept-Language': 'en-GB,en;q=0.5',
    'Connection': 'keep-alive',
    'Referer': 'https://www.amazon.co.uk/ref=nav_logo',
    'Sec-Fetch-Dest': 'image',
    'Sec-Fetch-Mode': 'no-cors',
    'Sec-Fetch-Site': 'same-origin',
    'Priority': 'u=4',
    'Pragma': 'no-cache',
    'Cache-Control': 'no-cache',
}

In [10]:
def get_soup(url):
    response = requests.get(url, headers=headers, cookies=cookies)

    if response.status_code != 200:
        print("Error in getting webpage")
        exit(-1)

    soup = BeautifulSoup(response.text, "lxml")
    return soup

In [11]:
scraped_reviews = []

In [12]:
def get_reviews(reviewUrl):

    soup = get_soup(reviewUrl)
    review_elements = soup.select("div.review")

    for item in review_elements:

        review = {
            "author": item.find('span', {'class': 'a-profile-name'}).text.strip() if item.find('span', {'class': 'a-profile-name'}) else '',
            "rating": item.find('i', {'data-hook': 'review-star-rating'}).text.strip()[0] if item.find('i', {'data-hook': 'review-star-rating'}) else '',
            "title": item.find('a', {'data-hook': 'review-title'}).text.strip().split('\n', 1)[1] if item.find('a', {'data-hook': 'review-title'}) else '',
            "content": item.find('span', {'data-hook': 'review-body'}).text.strip() if item.find('span', {'data-hook': 'review-body'}) else '',
            "date": item.find('span', {'data-hook': 'review-date'}).text.strip().split('on ', 1)[1]  if item.find('span', {'data-hook': 'review-date'}) else ''
        }
        print(review)
        scraped_reviews.append(review)

    return scraped_reviews

In [13]:
def totalPages(productUrl):
    soup = get_soup(productUrl)
    reviews = soup.find('div', {'data-hook':"cr-filter-info-review-rating-count"})
    print(reviews.text.strip().split(', ')[1].split(" ")[0])
    return int(reviews.text.strip().split(', ')[1].split(" ")[0].replace(',',''))

In [14]:
search_url = "https://www.amazon.co.uk/Yaheetech-Dining-Kitchen-Breakfast-Stools/product-reviews/B0C167TLF8/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber="
reviewUrl = search_url + str(3)
print(reviewUrl)
totalPage = totalPages(search_url)
print(totalPage)

for i in range(totalPage//10):
    print(f"Running for page {i+1}")
    time.sleep(3)
    try:
        reviewUrl = search_url + str(i+1)
        print(reviewUrl)
        get_reviews(reviewUrl)
    except Exception as e:
        print(e)
        
   
df = pd.DataFrame(scraped_reviews)
print(df)
df.to_csv("xboxControllerReview.csv")

https://www.amazon.co.uk/Yaheetech-Dining-Kitchen-Breakfast-Stools/product-reviews/B0C167TLF8/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=3
56
56
Running for page 1
https://www.amazon.co.uk/Yaheetech-Dining-Kitchen-Breakfast-Stools/product-reviews/B0C167TLF8/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=1
{'author': 'Sarah white', 'rating': '5', 'title': 'Great quality for a great price', 'content': 'I bought This abit weary but it turned out to be perfect for our kitchen right size for the space we had, sturdy and easy to put together', 'date': '7 June 2024'}
{'author': 'Tina', 'rating': '5', 'title': 'Sturdy and easy to assemble', 'content': 'This table and stools fits into my living room so neatly.  It looks good and seems very practical.  It is very sturdy and was easy and quick to assemble.  Very pleased.', 'date': '9 January 2024'}
{'author': 'KC', 'rating': '4', 'title': 'Brilliant little table', 'content': 'For t